# Web Scraping Sample Patient Unstructred Data

From the Agency for Healthcare Research and Quality.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import os

def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Part I: Web Scraping Tool
Creating a function that web scrapes from multiple links (this is the unstructured data), parses the HTML (this is the ETL), and stores in a pandas DataFrame. The DataFrame gets convered to a CSV and a job scheduler imports it to the cloud based database. 

In [3]:
def web_scrape_page(url):
    resp = requests.get(url)
    
    # All info from webpage scraped
    soup = BeautifulSoup(resp.text,'html.parser')
    
    # Scrap Patient Information
    table = soup.find_all('table')[2]
    cols = table.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    newlist = [word for line in cols for word in line.split(':')]
    data = []
    value = []

    for i in range(len(newlist)):
        if i%2!= 0:
            data.append(newlist[i])
        else:
            value.append(newlist[i])
    patient_information = pd.DataFrame({'value': value, 'data': data})
    
    
    # Scrap Patient Appointment Information
    h = []
    p = []
    for header in soup.find_all('h4'):
        h.append(header.text)
        para = header.find_next_sibling('p')
        p.append(para)

    clean_p = []
    for i in p:
        clean_p.append(str(i))

    clean_p2 = []
    for i in clean_p:
        clean_p2.append(remove_html_tags(i))

    appt_information = pd.DataFrame({'patient_information':h, 'patient_data':clean_p2})
    
    # Scrap Vitals 
    table = soup.find_all('table')[7]
    cols = table.find_all('td')
    cols = [ele.text.strip() for ele in cols]

    vitals = cols[0::3]
    results = cols[1::3]
    info = cols[2::3]

    results2 = []
    for i in results:
        results2.append(i.replace("\xa0", ""))

    vitals2 = []
    for i in vitals:
        vitals2.append(i.replace("\xa0", ""))

    medrec_vitals = pd.DataFrame({'vitals': vitals2, 'vitals_results': results2})


    # Scrap Test results
    table = soup.find_all('table')[10]
    cols = table.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    flowsheet = cols[2:]

    test = []
    results = []

    for i in range(len(flowsheet)):
        if i%2!= 0:
            results.append(flowsheet[i])
        else:
            test.append(flowsheet[i])
    medrec_tests = pd.DataFrame({'test': test, 'test_results': results})
    
    # Append all df to eachother
    new_cols = {x: y for x, y in zip(patient_information.columns, appt_information.columns)}
    df_out = appt_information.append(patient_information.rename(columns=new_cols))
    new_cols = {x: y for x, y in zip(df_out.columns, medrec_vitals.columns)}
    df_out2 = medrec_vitals.append(df_out.rename(columns=new_cols))
    new_cols = {x: y for x, y in zip(df_out2.columns, medrec_tests.columns)}
    final_df = medrec_tests.append(df_out2.rename(columns=new_cols))
    
    # Transpose and create new row
    final_df = final_df.T
    new_header = final_df.iloc[0] 
    final_df.columns = new_header 
    final_df = pd.DataFrame(final_df.iloc[1,:]).T
    final_df = final_df.rename(index={'test_results':0})
        
    return final_df

In [4]:
pg1 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-adam-pie.html")
pg2 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-n-bill-windows.html")
pg3 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-billy-gato.html")
pg4 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-john-donut.html")
pg5 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-steve-apple.html")
pg6 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-tom-gellato.html")

frames = [pg1, pg2, pg3, pg4, pg5, pg6]
med_recs = pd.concat(frames, sort=False)
med_recs = med_recs.reset_index(drop=True)

### Part II: Transform Tool
Take the resulting DataFrame and transform it to meet the needs of the target database using python.

In [7]:
# Transform to target db structure
final_df = med_recs.copy()
cleaned_df = final_df[['Patient ID','Name', 'Address', 'Gender', 'Email', 'Birth Date', 
          'Soc Sec No', 'Home Phone', 'HEIGHT\xa0 (in)', 'WEIGHT (lb)',
         'PULSE RATE (/min)', 'TEMPERATURE (deg F)', 'BP SYSTOLIC (mm Hg)',
         'BP DIASTOLIC (mm Hg)', 'CHOLESTEROL (mg/dL)', 'HDL (mg/dL)', 'LDL (mg/dL)', 'Problems']]

cleaned_df[['CustFirstName','CustLastName']] = cleaned_df.Name.str.split(expand=True) 
cleaned_df['CustMiddleInitial'] = np.nan
cleaned_df['Gender'] = np.where(cleaned_df['Gender'] == ' Male', 'M', 'F')
cleaned_df['Address'] = cleaned_df['Address'].str.replace('\n', ', ')
cleaned_df['Address'] = cleaned_df['Address'].str.replace('\t', '')
cleaned_df[['Street','City', 'State']] = cleaned_df['Address'].str.split(',',expand=True) 
cleaned_df['ZipCode'] = np.nan
cleaned_df['Problems'] = cleaned_df['Problems'].str.replace('\n', ',')

dieases = {'Diabetes Mellitus':[], 'Cirrhosis':[], 'Pneumonia':[], 'Alzheimer\'s disease':[], 
            'HIV disease':[], 'Cardiovascular disease':[], 'Hypertension':[]}
for i in cleaned_df['Problems']:
    for j in dieases:
        if j.lower() in str(i).lower():
            dieases[j].append('Yes') 
        else:
            dieases[j].append('No') 
            
dieases = pd.DataFrame(dieases)
cleaned_df = pd.concat([cleaned_df,dieases], axis = 1)
cleaned_df = cleaned_df.rename(columns={'Patient ID':'CustID', 'Email':'EmailAddress', 'Birth Date':'DOB', 
                           'Soc Sec No':'Ssn', 'Home Phone': 'ContactNumber', 'HEIGHT\xa0 (in)':'HEIGHT_IN',
                           'WEIGHT (lb)': 'Weight_LBS','PULSE RATE (/min)': 'PulseRate_per_min',
                           'TEMPERATURE (deg F)': 'Body_temp_deg_F', 
                           'BP SYSTOLIC (mm Hg)':'BP_SYSTOLIC_MM_HG',
                           'BP DIASTOLIC (mm Hg)':'BP_DIASTOLIC_MM_HG', 
                           'CHOLESTEROL (mg/dL)':'CHOLESTEROL_MG_PER_DL', 
                           'HDL (mg/dL)':'BLOODSUGARFASTING_MG_PER_DL', 
                           'LDL (mg/dL)': 'BLOODSUGARAFTERMEALS_MG_PER_DL',
                            'Diabetes Mellitus':'DIABETES_MELLITUS',
                             'Alzheimer\'s disease':'ALZHEIMERS_DISEASE',
                                       'HIV disease':'HIV_disease',
                                       'Cardiovascular disease':'Cardiovascular_disease'})

cleaned_df['CustID'] = cleaned_df['CustID'].str.replace('-', '')
cleaned_df['CustID'] = cleaned_df.CustID.str[4:]

cleaned_df['HEIGHT_IN'] = cleaned_df['HEIGHT_IN'].astype(float)
cleaned_df['Weight_LBS'] = cleaned_df['Weight_LBS'].astype(float)

cleaned_df['PREVIOUS_PLAN'] = ['Lifestyle Plus Coverage Plan', 'Sugar Rush Coverage Plan', 'Lifestyle Plus Coverage Plan', 'Heart Plus Coverage Plan', 'Sugar Rush Coverage Plan', 'Basic Coverage Plan']


return_df = cleaned_df[['CustID','CustFirstName','CustMiddleInitial','CustLastName','DOB','Ssn',
                       'Street','City', 'State', 'ZipCode', 'Gender', 'EmailAddress', 
                       'ContactNumber','HEIGHT_IN',
                       'Weight_LBS','PulseRate_per_min',
                       'Body_temp_deg_F', 
                       'BP_SYSTOLIC_MM_HG',
                       'BP_DIASTOLIC_MM_HG', 
                       'CHOLESTEROL_MG_PER_DL', 
                       'BLOODSUGARFASTING_MG_PER_DL', 
                       'BLOODSUGARAFTERMEALS_MG_PER_DL','DIABETES_MELLITUS', 'Cirrhosis', 'Pneumonia', 'ALZHEIMERS_DISEASE', 
                       'HIV_disease', 'Cardiovascular_disease', 'Hypertension', 'PREVIOUS_PLAN']]

In [8]:
return_df

,CustID,CustFirstName,CustMiddleInitial,CustLastName,DOB,Ssn,Street,City,State,ZipCode,...,BLOODSUGARFASTING_MG_PER_DL,BLOODSUGARAFTERMEALS_MG_PER_DL,DIABETES_MELLITUS,Cirrhosis,Pneumonia,ALZHEIMERS_DISEASE,HIV_disease,Cardiovascular_disease,Hypertension,PREVIOUS_PLAN
0,088888,Adam,NaN,Pie,08/08/1948,111-11-1111,1111 Donut Road,Fast Food,California,NaN,...,127,,Yes,No,No,No,No,No,Yes,Lifestyle Plus Coverage Plan
1,099999,Bill,NaN,Windows,09/09/1953,999-99-9999,9999 Computer Dr,Operating System,California,NaN,...,87,125,Yes,No,No,No,No,No,No,Sugar Rush Coverage Plan
2,055555,Billy,NaN,Gato,05/05/1955,555-55-5555,5555 Mountain Blvd,Animal,California,NaN,...,NaN,NaN,Yes,No,No,No,No,No,Yes,Lifestyle Plus Coverage Plan
3,011111,John,NaN,Donut,01/01/1935,111-11-1111,1111 Donut Road,Fast Food,California,NaN,...,NaN,NaN,Yes,No,No,No,No,No,Yes,Heart Plus Coverage Plan
4,022222,Steve,NaN,Apple,02/02/1945,222-22-2222,2222 Computer Dr,Laptop,California,NaN,...,87,125,Yes,No,No,No,No,No,No,Sugar Rush Coverage Plan
5,066666,Tom,NaN,Gellato,06/06/1938,666-66-6666,5555 Flavor Avenue,Ice Cream,California,NaN,...,102,125,Yes,No,No,No,No,No,No,Basic Coverage Plan


In [9]:
# Insert Statements
print("""

Insert into Customers values('088888', 'Adam', NULL, 'Pie', '1948/08/08','111-11-1111', '1111 Donut Road', 'Fast Food', 'California', NULL, 'M', NULL, '888-888-8888', '70.0', '190.0', '72', '98', '158', '90', NULL, '127', NULL, 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Lifestyle Plus Coverage Plan');

Insert into Customers values('099999', 'Bill', NULL, 'Windows', '1953/09/09', '999-99-9999', '9999 Computer Dr', 'Operating System', 'California', NULL, 'M', NULL, '999-999-9999', '70.0', '190.0', '72', '98', '128', '70', NULL, '87', '125', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Sugar Rush Coverage Plan');

Insert into Customers values('055555', 'Billy', NULL, 'Gato', '1955/05/05', '555-55-5555', '5555 Mountain Blvd', 'Animal', 'California', NULL, 'M', NULL, '555-555-5555', '65.0', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Lifestyle Plus Coverage Plan');

Insert into Customers values('011111', 'John', NULL, 'Donut', '1935/01/01', '111-11-1111', '1111 Donut Road', 'Fast Food', 'California', NULL, 'M', NULL, '000-000-0000', '74.0', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Heart Plus Coverage Plan');

Insert into Customers values('022222', 'Steve', NULL, 'Apple', '1945/02/02', '222-22-2222', '2222 Computer Dr', 'Laptop', 'California', NULL, 'M', NULL, '222-222-2222', '71.0', '191.0', '72', '98', '118', '70', NULL, '87', '125', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Sugar Rush Coverage Plan');

Insert into Customers values('066666', 'Tom', NULL, 'Gellato', '1938/06/06', '666-66-6666', '5555 Flavor Avenue', 'Ice Cream', 'California', NULL, 'M', NULL, '666-666-6666', '66.0', '195.0', '72', '98', '131', '94', NULL, '102', '125', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Basic Coverage Plan');
""")



Insert into Customers values('088888', 'Adam', NULL, 'Pie', '1948/08/08','111-11-1111', '1111 Donut Road', 'Fast Food', 'California', NULL, 'M', NULL, '888-888-8888', '70.0', '190.0', '72', '98', '158', '90', NULL, '127', NULL, 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Lifestyle Plus Coverage Plan');

Insert into Customers values('099999', 'Bill', NULL, 'Windows', '1953/09/09', '999-99-9999', '9999 Computer Dr', 'Operating System', 'California', NULL, 'M', NULL, '999-999-9999', '70.0', '190.0', '72', '98', '128', '70', NULL, '87', '125', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Sugar Rush Coverage Plan');

Insert into Customers values('055555', 'Billy', NULL, 'Gato', '1955/05/05', '555-55-5555', '5555 Mountain Blvd', 'Animal', 'California', NULL, 'M', NULL, '555-555-5555', '65.0', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Lifestyle Plus Coverage Plan');

Insert into Customers values('011111', 'John', NULL, 'Donut', '1935/01/01'

In [16]:
# Save return_df as csv
cwd = os.getcwd()
output_path = cwd + '/Med_Recs.csv'
return_df.to_csv(output_path, index = True)